In [1]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    import os
    os.chdir('/content')
    if not os.path.isdir('/content/EVA-2-Group/'):
        !git clone https://github.com/sambitdash/EVA-2-Group.git
    os.chdir('/content/EVA-2-Group/Session-19')
    !pwd
    
    !git config user.email "sambitdash@gmail.com"
    !git config user.name "Sambit Kumar Dash"
    !git config user.password "your password"
    !git status

Cloning into 'EVA-2-Group'...
remote: Enumerating objects: 1096, done.
remote: Counting objects: 100% (1096/1096), done.
remote: Compressing objects: 100% (1080/1080), done.
remote: Total 2192 (delta 13), reused 1087 (delta 7), pack-reused 1096
Receiving objects: 100% (2192/2192), 592.88 MiB | 35.35 MiB/s, done.
Resolving deltas: 100% (77/77), done.
Checking out files: 100% (2036/2036), done.
/content/EVA-2-Group/Session-19
On branch master
Your branch is up to date with 'origin/master'.

nothing to commit, working tree clean


In [0]:
import cv2
import os
import numpy as np
from os import listdir
from os.path import join

car_types = ['hatch', 'sedan']

def resize_image(img, size=(64,64)):

    h, w = img.shape[:2]

    if h == w: 
        return cv2.resize(img, size, cv2.INTER_AREA)

    dif = h if h > w else w

    interpolation = cv2.INTER_AREA if dif > (size[0]+size[1])//2 else cv2.INTER_CUBIC

    x_pos = (dif - w)//2
    y_pos = (dif - h)//2

    if len(img.shape) == 2:
        mask = np.zeros((dif, dif), dtype=img.dtype)
        mask[y_pos:y_pos+h, x_pos:x_pos+w] = img[:h, :w]
    else:
        c = img.shape[2]
        mask = np.zeros((dif, dif, c), dtype=img.dtype)
        mask[y_pos:y_pos+h, x_pos:x_pos+w, :] = img[:h, :w, :]

    return cv2.resize(mask, size, interpolation)

spath, dpath = join('data', 'cars'), join('data', 'norm')

if not os.path.isdir(dpath):
    os.mkdir(dpath)

for ct in car_types:
    sp, dp = join(spath, ct), join(dpath, ct)
    if not os.path.isdir(dp):
        os.mkdir(dp)
    for f in listdir(sp):
        sf, df = join(sp, f), join(dp, f)
        img = cv2.imread(sf)
        img = resize_image(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        cv2.imwrite(df, img)

In [0]:
%tensorflow_version 1.x

from tensorflow.keras import utils 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

(trainx, trainy), (testx, testy) = cifar10.load_data()
ntrain, rows, cols, ch =  trainx.shape
ntest, _, _, _         =  testx.shape
ncls = len(np.unique(trainy))
print(ncls)

trainx = trainx.astype('float32') / 255
testx = testx.astype('float32') / 255

trainx_mean = trainx.mean(axis=0)
trainx_std  = trainx.std(axis=0)

#print(trainx_mean, trainx_std)

datagen = ImageDataGenerator(zca_whitening=True)

datagen.fit(trainx)

iterate = datagen.flow(trainx, trainy, batch_size=len(trainx), shuffle=False)

trainx, trainy = iterate.next()

iterate = datagen.flow(testx, testy, batch_size=len(testx), shuffle=False)

testx, testy = iterate.next()

# trainx -= trainx_mean
# trainx /= trainx_std

# testx -= trainx_mean
# testx /= trainx_std

trainX, trainY = trainx, utils.to_categorical(trainy)
testX,  testY  = testx,  utils.to_categorical(testy)

min_pix, max_pix = trainX.min(), trainX.max()

print(min_pix, max_pix)
print(testX.min(), testX.max())

print(trainX.shape, trainY.shape)

In [0]:
BATCH_SIZE = 512
WT_DECAY   = 1e-4
LRFNEPOCH  = 4
MOMENTUM   = 0.9
EPOCHS     = 300
LEARNING_RATE = 0.01
ncls = 2

from tensorflow.keras.layers import BatchNormalization, Conv2D, Activation, MaxPool2D
from tensorflow.keras.layers import add, Input, Dense, Flatten, GlobalAvgPool2D
from tensorflow.keras.initializers import zeros

def ResConv(x, kernel=(3, 3), depth=32, maxpool=False):
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    if maxpool :
        x = MaxPool2D()(x)
    x = Conv2D(depth, kernel, padding='same', use_bias=False)(x)
    return x

def ResUnit(x, depth=32, maxpool=False):
    x = ResConv(x, depth=depth, maxpool=maxpool)
    x = ResConv(x, depth=depth)
    return x
    
def ResNetBlock(x, nunit, depth=32, maxpool=False, name="Block-1"):
    assert nunit > 0, "Ensure there are at least 1 unit in the ResNet Block"
    nunit -= 1
    if maxpool:
        xskip = Conv2D(depth, (1, 1), strides=2, use_bias=False)(x)
    else: 
        xskip = x
    x = add([ResUnit(x, depth=depth, maxpool=maxpool), xskip])
    if nunit >= 1:
        nunit -= 1
        for i in range(nunit):
            x = add([ResUnit(x, depth=depth), x])
        x = add([ResUnit(x, depth=depth), x], name=name)
    return x

# Returns latent vector of 512 bytes        
def ResNet18(x):
    x = Conv2D(64, (7, 7), strides=2, padding='same', use_bias=False)(x)
    x = MaxPool2D((3, 3),  strides=2, padding='same')(x)
    
    nunits   = (2, 2, 2, 2)
    maxpools = (False, True, True, True)
    depths   = (64, 128, 256, 512)
    
    for i in range(4):
        x = ResNetBlock(x, nunits[i], depth=depths[i], maxpool=maxpools[i], name="Block-"+str(i))
    x = GlobalAvgPool2D()(x)
    return x

# Returns latent vector of 256 bytes
def ResNet9(x):
    x = Conv2D(64, (7, 7), strides=2, padding='same', use_bias=False)(x)
    x = MaxPool2D((3, 3),  strides=2, padding='same')(x)
    
    nunits   = (2, 3, 2)
    maxpools = (False, True, True)
    depths   = (64, 128, 256)
    
    for i in range(3):
        x = ResNetBlock(x, nunits[i], depth=depths[i], maxpool=maxpools[i], name="Block-"+str(i))
    x = GlobalAvgPool2D()(x)
    return x

In [11]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.layers import Flatten
from tensorflow.keras import backend as K

def l2_weights(model):
    l2 = 0
    for layer in model.layers: 
        wt = layer.weights
        if len(wt) > 0:
            l2 += K.sum(K.pow(wt, 2))
    return l2

def reg_loss(model):
    def rloss(y_true, y_pred):
        return model.l2_reg*l2_weights(model)
    return rloss

def loss_with_regularization(model):
    def loss(y_true, y_pred):
        return categorical_crossentropy(y_true, y_pred) + reg_loss(model)(y_true, y_pred)
    return loss

def model_init(optimizer=None):
    xin = Input(shape=(64, 64, 3), name="Input")
    x = ResNet9(xin)
    x = Dense(ncls, use_bias=False)(x)
    y = Activation('softmax')(x)
    model = Model(xin, y)
    model.l2_reg = K.variable(value=WT_DECAY, dtype='float32', name='reg_loss')
    if optimizer is None:
        optimizer = SGD(lr=1e-3, momentum=MOMENTUM, nesterov=True)
    model.compile(optimizer=optimizer, loss=loss_with_regularization(model), metrics=['acc', reg_loss(model)])
    return model

model = model_init()
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_80 (Conv2D)              (None, 32, 32, 64)   9408        Input[0][0]                      
__________________________________________________________________________________________________
max_pooling2d_16 (MaxPooling2D) (None, 16, 16, 64)   0           conv2d_80[0][0]                  
__________________________________________________________________________________________________
batch_normalization_64 (BatchNo (None, 16, 16, 64)   256         max_pooling2d_16[0][0]           
____________________________________________________________________________________________

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = 8 #int(np.sqrt(s / r))
            h = 8 #int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = 0.0 #np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

datagen = ImageDataGenerator(
        fill_mode = 'constant',
        cval=0,
        width_shift_range=4,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=4,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        preprocessing_function=get_random_eraser(v_l=min_pix, v_h=max_pix, pixel_level=False)
)
datagen.fit(trainX)

NameError: ignored